In [63]:
import pandas as pd
import plotly.express as px
from plotly.io import show

from fastf1.ergast import Ergast

Year = 2023

In [64]:
ergast = Ergast()
races = ergast.get_race_schedule(Year)  # Races in year 2022
#ergast.get_constructor_info(2022)
results = []

In [50]:
ergast.get_race_results(season=Year, round=1).content[0]['constructorName']

0           Red Bull
1           Red Bull
2       Aston Martin
3            Ferrari
4           Mercedes
5       Aston Martin
6           Mercedes
7         Alfa Romeo
8     Alpine F1 Team
9           Williams
10        AlphaTauri
11          Williams
12      Haas F1 Team
13        AlphaTauri
14      Haas F1 Team
15        Alfa Romeo
16           McLaren
17    Alpine F1 Team
18           Ferrari
19           McLaren
Name: constructorName, dtype: object

In [65]:
for rnd, race in races['raceName'].items():
    print("Race name ",race)
    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_race_results(season=Year, round=rnd + 1)
    if len(temp.content) < 1:
        break
    else:
        temp = temp.content[0]
    

    # If there is a sprint, get the results as well
    sprint = ergast.get_sprint_results(season=Year, round=rnd + 1)
    if sprint.content and sprint.description['round'][0] == rnd + 1:
        temp = pd.merge(temp, sprint.content[0], on='constructorName', how='left')
        # Add sprint points and race points to get the total
        temp['points'] = temp['points_x'] + temp['points_y']
        temp.drop(columns=['points_x', 'points_y'], inplace=True)

    # Add round no. and grand prix name
    temp['round'] = rnd + 1
    temp['race'] = race.removesuffix(' Grand Prix')
    temp = temp[['round', 'race', 'constructorName', 'points']]  # Keep useful cols.
    results.append(temp)

# Append all races into a single dataframe
results = pd.concat(results)
races = results['race'].drop_duplicates()


Race name  Bahrain Grand Prix
Race name  Saudi Arabian Grand Prix
Race name  Australian Grand Prix
Race name  Azerbaijan Grand Prix
Race name  Miami Grand Prix
Race name  Monaco Grand Prix
Race name  Spanish Grand Prix
Race name  Canadian Grand Prix
Race name  Austrian Grand Prix
Race name  British Grand Prix
Race name  Hungarian Grand Prix
Race name  Belgian Grand Prix
Race name  Dutch Grand Prix
Race name  Italian Grand Prix
Race name  Singapore Grand Prix
Race name  Japanese Grand Prix
Race name  Qatar Grand Prix
Race name  United States Grand Prix
Race name  Mexico City Grand Prix
Race name  São Paulo Grand Prix


In [66]:
test = results.groupby(["round", "race", "constructorName"]).sum().reset_index()

In [67]:
results = test.pivot(index='constructorName', columns='race', values='points')
results
results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races
results

race,Bahrain,Saudi Arabian,Australian,Azerbaijan,Miami,Monaco,Spanish,Canadian,Austrian,British,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,Qatar,United States,Mexico City
constructorName,,,,,,,,,,,,,,,,,,,
Red Bull,36.0,112.0,114.0,43.0,102.0,34.0,34.0,37.0,41.0,43.0,26.0,25.0,44.0,25.0,68.0,44.0,14.0,38.0,98.0
Ferrari,0.0,64.0,72.0,12.0,48.0,3.0,22.0,10.0,10.0,27.0,20.0,27.0,16.0,12.0,26.0,14.0,37.0,10.0,48.0
Mercedes,18.0,22.0,40.0,16.0,48.0,25.0,15.0,8.0,20.0,18.0,16.0,27.0,20.0,23.0,42.0,22.0,16.0,33.0,36.0
McLaren,12.0,24.0,4.0,0.0,32.0,30.0,0.0,8.0,28.0,4.0,33.0,14.0,0.0,3.0,94.0,0.0,24.0,0.0,46.0
Aston Martin,27.0,42.0,44.0,23.0,24.0,6.0,20.0,19.0,3.0,2.0,4.0,0.0,15.0,18.0,18.0,15.0,0.0,14.0,12.0
Alpine F1 Team,0.0,6.0,0.0,2.0,20.0,0.0,4.0,16.0,0.0,0.0,3.0,1.0,6.0,21.0,12.0,6.0,8.0,5.0,20.0
Williams,0.0,0.0,0.0,1.0,0.0,4.0,6.0,4.0,0.0,6.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,6.0
AlphaTauri,1.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0
Alfa Romeo,2.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,2.0,0.0


In [68]:
import plotly.express as px
import webbrowser  # Import the webbrowser module to open the HTML file in the browser

# Assuming 'results' is your data, replace it with your actual dat
fig = px.imshow(
    results,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Points'}       # Change hover texts
)
fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')
fig.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig.update_layout(coloraxis_showscale=False)        # Remove legend
fig.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig.write_html('plot_teams.html')

# Open the HTML file in the default web browser
webbrowser.open('plot_teams.html')

True